In [40]:
%pylab inline
%config InlineBackend.figure_format = 'retina'

Populating the interactive namespace from numpy and matplotlib


In [102]:
import itertools, os; import pandas as pd; import numpy  as np
from pandas.tseries.offsets import *
import statsmodels.formula.api as smf
import glob
import xlsxwriter

In [2]:
os.chdir("/Users/abramo/Dropbox/Results X_N/")

In [208]:
dfs   = []
files = glob.glob("summary*")
for X, N in zip([1,1,1,2,2,2,5,5,5], [1,5,10]*3):
    file = 'summary_of_results_X%d_N%d.xlsx'%(X,N)
    te = pd.read_excel(file)
    te['N'] = N; te['X'] = "{0:d}%".format(X)
    dfs.append(te)

df = pd.concat(dfs)
ds = df.reset_index().set_index(['index', 'X', 'N']).unstack(level=1)["(1)"]
S  = ds.index.get_level_values(0).nunique()
for i in range(0, S): ds = pd.concat([ds.iloc[:i*4], pd.DataFrame(index=[('', '')]), ds.iloc[i*4:]])

st = ds.copy()
ds = ds.apply(lambda x : x.str.replace("*", ""))
ds = ds.astype(float)

st1 = st.apply(lambda x : x.str[-1:]=='*' )
st2 = st.apply(lambda x : x.str[-2:]=='**' )
st3 = st.apply(lambda x : x.str[-3:]=='***' )

In [209]:
writer = pd.ExcelWriter('Comparison.xlsx', engine='xlsxwriter')
ds.to_excel(writer, sheet_name='Sheet1')
workbook  = writer.book

worksheet = writer.sheets['Sheet1']

for i in range(0, S): worksheet.conditional_format('C%d:E%d'%(i*4+3,i*4+6), {
    'type'     : '2_color_scale',
    'min_color': '#ffffff',
    'max_color': '#99ddff',
})
    
worksheet.set_column('A:A', 30)
worksheet.set_zoom(175)

bold_cell = workbook.add_format()
bold_cell.set_bold(True)

non_bold_cell = workbook.add_format()
non_bold_cell.set_bold(False)

rows = ['C','D', 'E']
i = 0
j = 2
for key, row in ds.iterrows():
    for x in row:
        i+= 1
        y = ds .iloc[j-2][i%3]
        b = st1.iloc[j-2][i%3]
        if b : cf = bold_cell
        else : cf = non_bold_cell
        if not np.isnan(y): worksheet.write(rows[i%3]+str(j), y, cf)
        
    j+=1

writer.save()